### Clustering de palabras

In [6]:
import spacy
from spacy.tokens import Doc
import os
import numpy as np
from collections import defaultdict
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans, AgglomerativeClustering
import re

In [7]:
# regex tokenizer
PATTERN = r'''(?x)
   (?:Ley\sN[ºo°]*\s\d{1,}(?:\.\d+)*)                # Leyes como entidad
   | (?:[Aa]nexo[s]?[\sIVXLCDMy,]*[IVXLCDM]+)        # anexos
   | (?:Nota[\sA-Za-z()ºo°\.]+[\d|/]+)               # Notas como entidad
   | (?:Decreto[A-Zºo°\sa-z]+[\d|/]+)                # Decretos como entidad
   | (?:[Aa]rt[ií]culo[A-Z|\sºo°]+\d+º*)             # Articulos como entidad
   | (?:[Aa]rt\.*[\s+\d+]+º*)                        # abreviacion de articulo
   | (?:Resoluci[óo]n[A-Zºo°\sa-z]+[\d/]+)           # Resoluciones como entidad
   | (?:Disposici[óo]n[A-Zºo°\sa-z]+[\d/]+)          # Disposicion como entidad
   | (?:Expediente[A-Zºo°\s]+[\d/]+)                 # Expediente como entidad
   | (?:punto\s[\d\.]+)                              # punto x.x.x. como entidad
   | (?:\d{1,2}[\sa-z]+\d{4})                        # fechas
   | (?:[A-Z][a-záéíóú]+\s[A-Z]\.\s[A-Z][a-záéíóú]+) # entidades humanas Fulano M. Mengano
   | (?:MINISTERIO[\sA-Z,]*[A-Z]+)                   # Ministerios como entidad
   | (?:REPUBLICA[\sDE]*[A-Z]+)                      # Republica como entidad
   | (?:SECRETARIA[\sA-Z,]*[A-Z]+)                   # Secretaria como entidad
   | (?:SERVICIO[\sA-Z,]*[A-Z]+)                     # Servicios como entidad
   | (?:DIRECCION[\sA-Z,]*[A-Z]+)                    # Direccion como entidad
   | \w+(?:-\w+)*                                    # palabras con '-' opcional
   | \.\.\.                                          # ...
   | [][.,;"'?():-_`]
   | (?:\d+)                                         # numeros
   | (?:[.\n])                                       # punto y aparte
'''

# regex named entity
IS_NE = r'''(?x)
   (?:Ley\sN[ºo°]*\s\d{1,}(?:\.\d+)*)                # Leyes como entidad
   | (?:Nota[\sA-Za-z()ºo°\.]+[\d|/]+)               # Notas como entidad
   | (?:Decreto[A-Zºo°\sa-z]+[\d|/]+)                # Decretos como entidad
   | (?:Resoluci[óo]n[A-Zºo°\sa-z]+[\d/]+)           # Resoluciones como entidad
   | (?:Disposici[óo]n[A-Zºo°\sa-z]+[\d/]+)          # Disposicion como entidad
   | (?:Expediente[A-Zºo°\s]+[\d/]+)                 # Expediente como entidad
   | (?:punto\s[\d\.]+)                              # punto x.x.x. como entidad
   | (?:[A-Z][a-záéíóú]+\s[A-Z]\.\s[A-Z][a-záéíóú]+) # entidades humanas Fulano M. Mengano
   | (?:MINISTERIO[\sA-Z,]*[A-Z]+)                   # Ministerios como entidad
   | (?:REPUBLICA[\sDE]*[A-Z]+)                      # Republica como entidad
   | (?:SECRETARIA[\sA-Z,]*[A-Z]+)                   # Secretaria como entidad
   | (?:SERVICIO[\sA-Z,]*[A-Z]+)                     # Servicios como entidad
   | (?:DIRECCION[\sA-Z,]*[A-Z]+)                    # Direccion como entidad
'''

In [8]:
text = [i for i in os.listdir('corpus') if i.endswith('txt')]
raw_text = ''
for txt in text:
    with open(f'corpus/{txt}', 'r') as f:
        raw_text += f.read() + '\n'


In [9]:
class RegexTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = re.findall(IS_NE, text)
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

In [14]:
class InfolegFeatureExtractor:
    def __init__(self, tokenizer=None, spacy_model='es_core_news_sm'):
        self._nlp = spacy.load(spacy_model)
        self.vocabulary = []
        self._nlp.tokenizer = tokenizer(self._nlp.vocab)
        
    def _processing(self, text):
        return [t for t in self._nlp(text)]
            
    def _make_feature_dict(self, doc):
        document_features = []
        
        for i, token in enumerate(doc):
            features = {
                'word': token.text,
                'pword': '<s>',
                'nword': '</s>',
                'pword_tag': 'first',
                'nword_tag': 'last',
                'word_tag': token.pos_,
            }
            if 0 < i:
                features['pword'] = doc[i-1].text
                features['pword_tag'] = doc[i-1].pos_
            if i < len(doc) - 1:
                features['nword'] = doc[i+1].text
                features['nword_tag'] = doc[i+1].pos_

            document_features.append(features)
            self.vocabulary.append(token.text)
        return document_features
    
    def fit(self, X, y=None):
        return self 
    
    def _merge_features(self, feats):
        features = defaultdict(list)
        merged_dicts = []
        for feature in feats:
            features[feature.pop('word')].append(feature)
        for f in features.values():
            dd = defaultdict(str)
            for d in f:
                for key, value in d.items():
                    dd[key] += ' ' + value
            merged_dicts.append(dd)
        return merged_dicts

        
    def transform(self, text, y=None):
        tokens = self._processing(text)
        features = self._make_feature_dict(tokens)
        merged_features = self._merge_features(features)
        return np.array(merged_features)


In [15]:
class ExtendedKMeans(KMeans):
    def get_clusters(self, vocabulary):
        clusters = defaultdict(set)
        added_words = set()
        for i, label in enumerate(self.labels_):
            if vocabulary[i] not in added_words:
                clusters[label].add(vocabulary[i])
                added_words.add(vocabulary[i])
        return dict(clusters).values()

In [16]:
## Configs for each pipeline step

f_e_config = {
    'tokenizer' : RegexTokenizer,
}

vectorizer_config = {
        'sparse': False,
}

svd_config = {
        'n_components': 5,
        'n_iter': 5,
}
kmeans_config = {
        'n_clusters': 15,
        'init': 'k-means++',
        'precompute_distances': False,
}


In [17]:
pipeline = Pipeline([
                ('preprocessor', InfolegFeatureExtractor(**f_e_config)),
                ('vect', DictVectorizer(**vectorizer_config)),
                ('svd', TruncatedSVD(**svd_config)),
                ('kmeans', ExtendedKMeans(**kmeans_config)),
    
            ])
pipeline.fit(raw_text)

Pipeline(memory=None,
     steps=[('preprocessor', <__main__.InfolegFeatureExtractor object at 0x7f5dd35c59e8>), ('vect', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)), ('svd', TruncatedSVD(algorithm='randomized', n_components=5, n_iter=5,
       random_state=None, tol=0.0)), (...nit=10, n_jobs=None, precompute_distances=False,
        random_state=None, tol=0.0001, verbose=0))])

In [18]:
vocabulary = pipeline.named_steps['preprocessor'].vocabulary
clusters = pipeline.named_steps['kmeans'].get_clusters(vocabulary)

In [19]:
for i in clusters:
    print(i)

{'punto 3.2.', 'Resolución N° 80/2009', 'Ley Nº 24.240'}
{'Decretos Nº 180', 'Decreto Nº 181/04', 'Resolución Nº 736', 'Ley N° 26.993', 'Resolución Nº 834/02', 'Ley Nº 22.802', 'Decreto Nº 181/2004', 'Resolución Nº 834', 'SERVICIO NACIONAL DE SANIDAD Y CALIDAD AGROALIMENTARIA', 'SECRETARIA DE ENERGIA', 'punto 9.4.2.', 'Expediente ENARGAS Nº 8043', 'punto 9.4.2.2.', 'Notas ENRG/', 'MINISTERIO DE ECONOMIA', 'Decreto N° 27/2018', 'Decreto Nº 180', 'Resolución 208', 'Disposición Nº 27', 'Decreto Nº 583', 'punto 9.4.2.7.', 'Resolución Nº 265', 'Ley N° 27.266', 'Resolución SECRETARIA DE ENERGIA Nº 265/2004', 'punto 3.1', 'Nota ENRG Nº 2975/2004', 'SERVICIOS PUBLICOS DOMICILIARIOS\nARTICULO', 'Decreto N° 652', 'Resolución Nº 91/11', 'Ley Nº 25.972', 'MINISTERIO DE PLANIFICACION FEDERAL, INVERSION PUBLICA Y SERVICIOS', 'Resolución MPFIPyS Nº 208/2004', 'Ley No 24.076', 'MINISTERIO DE ECONOMIA Y FINANZAS PUBLICAS', 'MINISTERIO DE PRODUCCION', 'punto 9.4.2.3.', 'Decreto PEN Nº 181/04', 'Ley Nº 2